In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import csv


from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import json
import tensorflow as tf

from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
config.gpu_options.per_process_gpu_memory_fraction = 0.3
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config)) 


/home/tony/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_dataset = pd.read_csv('train.csv')
test_dataset = pd.read_csv('test.csv')

X = train_dataset.iloc[:, 5]


X_ = test_dataset.iloc[:, 4]

dataset_text = X.append(X_)

train_objs_num = len(X)
print(train_objs_num)
print(dataset_text.shape)

1280000
(1600000,)


In [3]:
# Data preprocessing
data_pre = dataset_text
data_pre = data_pre.apply(lambda x:x.lower())
data_pre = data_pre.apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x)) #Remove all characters not a~z and 0~9

tokenizer = Tokenizer(num_words = 2500, split=' ')
tokenizer.fit_on_texts(data_pre.values)
data_pre = tokenizer.texts_to_sequences(data_pre.values)
data_pre = pad_sequences(data_pre)

print("Done")

Done


In [4]:
print(data_pre.shape)
print(train_objs_num)

X = data_pre[:train_objs_num]
test = data_pre[train_objs_num:]

y = train_dataset.iloc[:, 0]

print(X.shape)
print(y.shape)
print(test.shape)
X

(1600000, 40)
1280000
(1280000, 40)
(1280000,)
(320000, 40)


array([[   0,    0,    0, ..., 1506, 1507,  710],
       [   0,    0,    0, ...,  369,   89,   49],
       [   0,    0,    0, ...,  286, 2067,   28],
       ...,
       [   0,    0,    0, ...,  346,   10,  434],
       [   0,    0,    0, ...,    4,  296,  159],
       [   0,    0,    0, ...,  115,  403,  200]], dtype=int32)

In [10]:
embed_dim = 128
lstm_out = 350
batch_size= 1000

model = Sequential()
model.add(Embedding(2500, embed_dim,input_length = X.shape[1], dropout=0.1))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(2,activation='sigmoid'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

Y = pd.get_dummies(y).values
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size = 0.20, random_state = 36)

#Here we train the Network.

model.fit(X_train, Y_train, batch_size = batch_size, epochs = 10,  verbose = 1)

# Measuring score and accuracy on validation set

score,acc = model.evaluate(X_valid, Y_valid, verbose = 2, batch_size = batch_size)
print("Logloss score: %.2f" % (score))
print("Validation set Accuracy: %.2f" % (acc))

/home/tony/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  
/home/tony/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(350, dropout=0.2, recurrent_dropout=0.2)`
  import sys


Epoch 1/10
1024000/1024000 [==============================] - 102s 100us/step - loss: 0.4619 - acc: 0.7796
Epoch 2/10
1024000/1024000 [==============================] - 104s 102us/step - loss: 0.4284 - acc: 0.8003
Epoch 3/10
1024000/1024000 [==============================] - 106s 103us/step - loss: 0.4187 - acc: 0.8058
Epoch 4/10
1024000/1024000 [==============================] - 107s 104us/step - loss: 0.4117 - acc: 0.8093
Epoch 5/10
1024000/1024000 [==============================] - 107s 105us/step - loss: 0.4053 - acc: 0.8127
Epoch 6/10
1024000/1024000 [==============================] - 108s 105us/step - loss: 0.3994 - acc: 0.8159
Epoch 7/10
1024000/1024000 [==============================] - 108s 106us/step - loss: 0.3939 - acc: 0.8191
Epoch 8/10
1024000/1024000 [==============================] - 108s 106us/step - loss: 0.3884 - acc: 0.8222
Epoch 9/10
1024000/1024000 [==============================] - 108s 105us/step - loss: 0.3828 - acc: 0.8250
Epoch 10/10
1024000/1024000 [========

In [11]:
pred = model.predict(test, verbose=1)

320000/320000 [==============================] - 163s 510us/step


In [7]:
pred
Y_valid

array([[1, 0],
       [0, 1],
       [1, 0],
       ...,
       [1, 0],
       [1, 0],
       [0, 1]], dtype=uint8)

In [12]:
with open('output.csv', 'w+') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['ID', 'Sentiment'])
    for i in range(len(pred)):
        writer.writerow([i, pred[i][1]*4])

print ("Done")

Done
